# Weather App | Development Notebook

In [1]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import openai
import asyncio
import nest_asyncio
from PIL import Image as PILImage
from IPython.display import Markdown, display, Image, HTML

os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from typing import List, Dict, Any
import uuid
from urllib.parse import urlparse
import base64
import lancedb
import requests

In [3]:
from duckduckgo_search import DDGS

In [4]:
results = DDGS().news(keywords="austin, tx", timelimit="d", max_results=3)

c:\Users\pdoub\opinionated-rag\.venv\lib\site-packages\curl_cffi\aio.py:204: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [5]:
results

[{'date': '2024-05-05T04:16:50+00:00',
  'title': '2024 Texas election live updates: May 4 elections underway in Austin area',
  'body': 'Election day in Texas has arrived.             Voters will head to the polls on May 4 to weigh in on candidates and propositions across Central Texas, including the Travis Central Appraisal District board of directors and a $649.',
  'url': 'https://www.msn.com/en-us/news/us/2024-texas-election-live-updates-get-latest-unofficial-voting-results-for-austin-area/ar-AA1o8nL9',
  'image': 'https://img-s-msn-com.akamaized.net/tenant/amp/entityid/BB1jnVTx.img?w=2000&h=1333&m=4&q=75',
  'source': 'Austin American-Statesman on MSN.com'},
 {'date': '2024-05-04T05:20:00+00:00',
  'title': 'Austin City Council at odds with Texas AG over transgender protections',
  'body': 'The Austin City Council passed a resolution ensuring transgender people may receive gender reassignment therapy.',
  'url': 'https://www.msn.com/en-us/news/other/austin-city-council-at-odds-wi

In [9]:
from pydantic import BaseModel, model_validator, field_validator
from typing import Optional, List
from datetime import datetime, timedelta
import requests
import os

def celsius_to_fahrenheit(temp_celsius: float) -> float:
    return (temp_celsius * 9/5) + 32

class WeatherData(BaseModel):
    temp: float
    temp_max: float
    temp_min: float
    feels_like: float
    description: str
    icon: str
    wind_speed: float
    wind_direction: int
    humidity: int
    rain: str
    cloud_cover: int
    sunset_local: str
    city_name: str
    date_stamp: str

    def __str__(self):
        return (
            f"{self.date_stamp}\n"
            f"In {self.city_name}, the weather is currently:\n"
            f"Status: {self.description.title()}\n"
            f"Wind speed: {self.wind_speed} m/s, direction: {self.wind_direction}°\n"
            f"Humidity: {self.humidity}%\n"
            f"Temperature: \n"
            f"  - Current: {self.temp}°F\n"
            f"  - High: {self.temp_max}°F\n"
            f"  - Low: {self.temp_min}°F\n"
            f"  - Feels like: {self.feels_like}°F\n"
            f"Rain: {self.rain if self.rain else 'No rain'}\n"
            f"Cloud cover: {self.cloud_cover}%"
        )
    
    @property
    def to_markdown(self):
        return (
            f"{self.date_stamp}\n\n"
            f"The weather in **{self.city_name}** is currently:\n\n"
            f"Status: {self.description.title()}\n\n"
            f"Wind speed: {self.wind_speed} m/s, direction: {self.wind_direction}°\n\n"
            f"Humidity: {self.humidity}%\n\n"
            f"Temperature: \n\n"
            f"  - Current: {self.temp}°F\n"
            f"  - High: {self.temp_max}°F\n"
            f"  - Low: {self.temp_min}°F\n"
            f"  - Feels like: {self.feels_like}°F\n\n"
            f"Rain: {self.rain if self.rain else 'No rain'}\n\n"
            f"Cloud cover: {self.cloud_cover}%"
        )

def fetch_weather(search_query: str, search_type: str = "city") -> WeatherData:
    API_key = os.getenv("OPENWEATHERMAP_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    
    if search_type == "city":
        final_url = f"{base_url}appid={API_key}&q={search_query}"
    elif search_type == "zip":
        final_url = f"{base_url}appid={API_key}&zip={search_query}"
    else:
        raise ValueError(f"Invalid search type: {search_type}. Must be either 'city' or 'zip'.")
    
    owm_response_json = requests.get(final_url).json()
    
    sunset_utc = datetime.fromtimestamp(owm_response_json["sys"]["sunset"])
    sunset_local = sunset_utc.strftime("%I:%M %p")
    
    temp_celsius = owm_response_json["main"]["temp"] - 273.15
    temp_max_celsius = owm_response_json["main"]["temp_max"] - 273.15
    temp_min_celsius = owm_response_json["main"]["temp_min"] - 273.15
    temp_feels_like_celsius = owm_response_json["main"]["feels_like"] - 273.15

    temp_fahrenheit = round(celsius_to_fahrenheit(temp_celsius), 2)
    temp_max_fahrenheit = round(celsius_to_fahrenheit(temp_max_celsius), 2)
    temp_min_fahrenheit = round(celsius_to_fahrenheit(temp_min_celsius), 2)
    temp_feels_like_fahrenheit = round(celsius_to_fahrenheit(temp_feels_like_celsius), 2)

    rain = owm_response_json.get("rain", "No rain")
    
    owm_dict = {
        "temp": temp_fahrenheit,
        "temp_max": temp_max_fahrenheit,
        "temp_min": temp_min_fahrenheit,
        "feels_like": temp_feels_like_fahrenheit,
        "description": owm_response_json["weather"][0]["description"],
        "icon": owm_response_json["weather"][0]["icon"],
        "wind_speed": owm_response_json["wind"]["speed"],
        "wind_direction": owm_response_json["wind"]["deg"],
        "humidity": owm_response_json["main"]["humidity"],
        "rain": rain,
        "cloud_cover": owm_response_json["clouds"]["all"],
        "sunset_local": sunset_local,
        "city_name": owm_response_json["name"],
        "date_stamp": datetime.utcnow().strftime("%A, %B %d, %Y")
    }
    
    return WeatherData(**owm_dict)

In [11]:
city = "austin"
weather = fetch_weather('78669', search_type='zip')
print(weather)

Sunday, May 05, 2024
In Spicewood, the weather is currently:
Status: Overcast Clouds
Wind speed: 2.68 m/s, direction: 360°
Humidity: 87%
Temperature: 
  - Current: 72.73°F
  - High: 74.07°F
  - Low: 71.35°F
  - Feels like: 73.8°F
Rain: No rain
Cloud cover: 100%


In [12]:
import requests

def get_local_alerts(max_alerts: int = 5) -> str:
    """
    Get local weather alerts for the current IP address location.

    Args:
        max_alerts (int): Maximum number of alerts to include in the output. Defaults to 5.

    Returns:
        str: Markdown-formatted string containing the weather alerts.
    """
    ip_address = requests.get('http://api.ipify.org').text
    geo_data = requests.get(f'http://ip-api.com/json/{ip_address}').json()
    lat = geo_data['lat']
    lon = geo_data['lon']
    response = requests.get(f'https://api.weather.gov/alerts?point={lat},{lon}').json()
    markdown_output = ""
    for x in response['features'][:max_alerts]:
        markdown_output += f"# {x['properties']['headline']}\n\n"
        markdown_output += f"### {x['properties']['areaDesc']}\n\n"
        markdown_output += f"{x['properties']['description']}\n\n"
        markdown_output += "---\n\n"
    return markdown_output

In [14]:
alerts = get_local_alerts(2)
Markdown(alerts)

# Severe Thunderstorm Watch issued May 5 at 12:02AM CDT until May 5 at 2:00AM CDT by NWS Austin/San Antonio TX

### Bandera, TX; Blanco, TX; Burnet, TX; Comal, TX; Edwards, TX; Gillespie, TX; Hays, TX; Kendall, TX; Kerr, TX; Kinney, TX; Llano, TX; Maverick, TX; Real, TX; Travis, TX; Williamson, TX

SEVERE THUNDERSTORM WATCH 185 REMAINS VALID UNTIL 2 AM CDT EARLY
THIS MORNING FOR THE FOLLOWING AREAS

IN TEXAS THIS WATCH INCLUDES 15 COUNTIES

IN SOUTH CENTRAL TEXAS

BANDERA               BLANCO                BURNET
COMAL                 EDWARDS               GILLESPIE
HAYS                  KENDALL               KERR
KINNEY                LLANO                 MAVERICK
REAL                  TRAVIS                WILLIAMSON

THIS INCLUDES THE CITIES OF AUSTIN, BANDERA, BLANCO, BOERNE,
BRACKETTVILLE, BURNET, EAGLE PASS, FREDERICKSBURG, GEORGETOWN,
KERRVILLE, LEAKEY, LLANO, NEW BRAUNFELS, ROCKSPRINGS,
AND SAN MARCOS.

---

# Severe Thunderstorm Watch issued May 4 at 9:39PM CDT until May 5 at 2:00AM CDT by NWS Austin/San Antonio TX

### Bandera, TX; Blanco, TX; Burnet, TX; Comal, TX; Edwards, TX; Gillespie, TX; Hays, TX; Kendall, TX; Kerr, TX; Kinney, TX; Llano, TX; Maverick, TX; Real, TX; Travis, TX; Williamson, TX

SEVERE THUNDERSTORM WATCH 185 REMAINS VALID UNTIL 2 AM CDT SUNDAY
FOR THE FOLLOWING AREAS

IN TEXAS THIS WATCH INCLUDES 15 COUNTIES

IN SOUTH CENTRAL TEXAS

BANDERA               BLANCO                BURNET
COMAL                 EDWARDS               GILLESPIE
HAYS                  KENDALL               KERR
KINNEY                LLANO                 MAVERICK
REAL                  TRAVIS                WILLIAMSON

THIS INCLUDES THE CITIES OF AUSTIN, BANDERA, BLANCO, BOERNE,
BRACKETTVILLE, BURNET, EAGLE PASS, FREDERICKSBURG, GEORGETOWN,
KERRVILLE, LEAKEY, LLANO, NEW BRAUNFELS, ROCKSPRINGS,
AND SAN MARCOS.

---

